# Q-Learning Experiments for Irrigation Scheduling

Notebook for running and experimenting with tabular Q-learning.

## Setup

In [13]:
import numpy as np
from irrigation_env import IrrigationEnv
from irr_Qtable import train_q_learning, discretize_state, get_state_space_size, N_ACTIONS

In [14]:
# Create environment instance
env = IrrigationEnv(
    max_et0=8.0,
    max_rain=50.0,
    et0_range=(2.0, 8.0),
    rain_range=(0.0, 40.0),
    episode_length=90,
)

print(f"Environment created")
print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Environment created
Action space: Discrete(3)
Observation space: Dict('crop_stage': Discrete(3), 'et0': Box(0.0, 1.0, (1,), float32), 'rain': Box(0.0, 1.0, (1,), float32), 'soil_moisture': Box(0.0, 1.0, (1,), float32))


## Training

In [15]:
# Training parameters
n_episodes = 1000
alpha = 0.1
gamma = 0.99
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 0.995
n_soil_bins = 12

print(f"Training Q-learning agent for {n_episodes} episodes...")
print(f"State space size: {get_state_space_size(n_soil_bins)}")
print(f"Action space size: {N_ACTIONS}")

Training Q-learning agent for 1000 episodes...
State space size: 144
Action space size: 3


In [16]:
# Train Q-learning
Q = train_q_learning(
    env=env,
    n_episodes=n_episodes,
    alpha=alpha,
    gamma=gamma,
    epsilon_start=epsilon_start,
    epsilon_end=epsilon_end,
    epsilon_decay=epsilon_decay,
    n_soil_bins=n_soil_bins,
)

print("\nTraining complete!")
print(f"Q-table shape: {Q.shape}")
print(f"Non-zero entries: {np.count_nonzero(Q)}/{Q.size}")


Training complete!
Q-table shape: (144, 3)
Non-zero entries: 72/432


## Inspection

In [17]:
# Basic Q-table statistics
print("Q-table Statistics:")
print(f"Shape: {Q.shape}")
print(f"Min Q-value: {Q.min():.4f}")
print(f"Max Q-value: {Q.max():.4f}")
print(f"Mean Q-value: {Q.mean():.4f}")
print(f"Std Q-value: {Q.std():.4f}")
print(f"\nNon-zero entries: {np.count_nonzero(Q)}/{Q.size} ({100*np.count_nonzero(Q)/Q.size:.1f}%)")

Q-table Statistics:
Shape: (144, 3)
Min Q-value: -1.5000
Max Q-value: 0.0000
Mean Q-value: -0.1025
Std Q-value: 0.3070

Non-zero entries: 72/432 (16.7%)


In [18]:
# Examine action preferences across all states
best_actions = np.argmax(Q, axis=1)
action_counts = np.bincount(best_actions, minlength=N_ACTIONS)

print("Action preferences (greedy policy):")
for action_idx, count in enumerate(action_counts):
    print(f"  Action {action_idx}: {count} states ({100*count/Q.shape[0]:.1f}%)")

Action preferences (greedy policy):
  Action 0: 144 states (100.0%)
  Action 1: 0 states (0.0%)
  Action 2: 0 states (0.0%)


In [19]:
# Test learned policy on a single episode
obs, info = env.reset(seed=123)
total_reward = 0.0
done = False
step_count = 0

print("Testing learned policy:")
while not done and step_count < 10:
    state = discretize_state(obs, n_soil_bins)
    action = np.argmax(Q[state])
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    total_reward += reward
    
    if step_count < 5:
        print(f"  Step {step_count+1}: state={state}, action={action}, reward={reward:.3f}, SM={obs['soil_moisture'][0]:.3f}")
    
    step_count += 1

print(f"\nTotal reward (first {step_count} steps): {total_reward:.3f}")

Testing learned policy:
  Step 1: state=75, action=0, reward=0.000, SM=0.643
  Step 2: state=85, action=0, reward=0.000, SM=0.770
  Step 3: state=111, action=0, reward=0.000, SM=1.000
  Step 4: state=135, action=0, reward=0.000, SM=1.000
  Step 5: state=133, action=0, reward=0.000, SM=1.000

Total reward (first 10 steps): 0.000


## Continued Training

In [22]:
# Reload module to pick up changes to train_q_learning
import importlib
import irr_Qtable
importlib.reload(irr_Qtable)
from irr_Qtable import train_q_learning, discretize_state, get_state_space_size, N_ACTIONS

print("Module reloaded")

Module reloaded


In [26]:
# Continue training from existing Q-table
n_additional_episodes = 500

print(f"Continuing training for {n_additional_episodes} additional episodes...")
print(f"Initial Q-table stats: min={Q.min():.4f}, max={Q.max():.4f}, mean={Q.mean():.4f}")

Q = train_q_learning(
    env=env,
    n_episodes=n_additional_episodes,
    alpha=alpha,
    gamma=gamma,
    epsilon_start=0.1,  # Lower exploration for continued training
    epsilon_end=0.01,
    epsilon_decay=0.99,
    n_soil_bins=n_soil_bins,
    Q_init=Q,
)

print("\nContinued training complete!")
print(f"Updated Q-table stats: min={Q.min():.4f}, max={Q.max():.4f}, mean={Q.mean():.4f}")

Continuing training for 500 additional episodes...
Initial Q-table stats: min=-1.5000, max=0.0000, mean=-0.1072

Continued training complete!
Updated Q-table stats: min=-1.5000, max=0.0000, mean=-0.1097


In [ ]:
# Compare action preferences after continued training
best_actions_updated = np.argmax(Q, axis=1)
action_counts_updated = np.bincount(best_actions_updated, minlength=N_ACTIONS)

print("Action preferences after continued training:")
for action_idx, count in enumerate(action_counts_updated):
    print(f"  Action {action_idx}: {count} states ({100*count/Q.shape[0]:.1f}%)")